<a href="https://colab.research.google.com/github/zsert1/sparta_coding_club_AI/blob/main/%ED%83%90%EC%83%89%EC%A0%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_(EDA)_%EB%B0%8F_%EC%A0%84%EC%B2%98%EB%A6%AC(%EC%8A%A4%ED%8C%8C%EB%A5%B4%ED%83%80_%EC%BD%94%EB%94%A9_%EC%9E%AC%EC%A7%81%EC%9E%90_%EA%B5%90%EC%9C%A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

금일 데이터 셋:
채무 불이행 데이터는 은행에서 대출 승인 여부 등을 결정할 때 도움을
받기 위해 구축 된 데이터로, 은행의 주요 수익원인 대출을 신규 고객에 대해 진행할지 말지
(해당 고객이 대출을 잘 상환할 고객이면 진행/그렇지 않으면 미진행) 결정할 때,
과거의 고객 정보 데이터 기반으로 머신러닝/딥러닝을 통해 푸는 도움을 제공하는 학습을 하기위한 데이터

## Libary Definition

In [1]:
import pandas as pd
import numpy as np

In [17]:
data_path='./sample_data/Loan_Default.csv'
num_sample=10
#
missing_value='remove'

# 데이터 전처리 시에 삭제 플래그 맟 대체
missing_value_flag = 'remove' # if flag is 'replace', missing values are replaced with the mode, etc.
# 데이터 전처리 시에 스케일링 플래그
scaling_flag = 'min-max' # if flag is 'standardization', numerical values are scaled by standardization

## 데이터 불러오기

In [15]:
# 키값이 중요하지 않은 경우이기 때무에 일반적으로 제거
eda_df=pd.read_csv(data_path)
eda_df.head(num_sample)
# 줄의 갯수
print('# of data: {}'.format(eda_df.shape[0]))
# 컬럼의 갯수
print('# of features: {}\n'.format(eda_df.shape[1]))
print('feature name/dtype')
# 숫자 데이터의 갯수
numerical_count = 0
# 숫자가 아닌 데이터의 갯수
categorical_count = 0
for feature_name, feature_type in zip(eda_df.columns, eda_df.dtypes):
  if np.issubdtype(feature_type, np.number):
    numerical_count += 1
  else:
    categorical_count += 1
  print('{}: {}'.format(feature_name, feature_type))

print('\n # of numerical item: {}/# of categorical item: {}'.format(numerical_count, categorical_count))
# 표준편차 분위 수 나오는 함수
eda_df.describe()

label_df = eda_df['Status']
label_dict = label_df.value_counts()
print('label shape: {}'.format(label_df.shape))
print('\nnum of negative label: {}/num of positive label: {}'.format(label_dict[0], label_dict[1]))
print('% of negative label: {}/% of positive label: {}'.format(label_dict[0] / label_df.shape[0] * 100, label_dict[1] / label_df.shape[0] * 100))

# of data: 148670
# of features: 34

feature name/dtype
ID: int64
year: int64
loan_limit: object
Gender: object
approv_in_adv: object
loan_type: object
loan_purpose: object
Credit_Worthiness: object
open_credit: object
business_or_commercial: object
loan_amount: int64
rate_of_interest: float64
Interest_rate_spread: float64
Upfront_charges: float64
term: float64
Neg_ammortization: object
interest_only: object
lump_sum_payment: object
property_value: float64
construction_type: object
occupancy_type: object
Secured_by: object
total_units: object
income: float64
credit_type: object
Credit_Score: int64
co-applicant_credit_type: object
age: object
submission_of_application: object
LTV: float64
Region: object
Security_Type: object
Status: int64
dtir1: float64

 # of numerical item: 13/# of categorical item: 21
label shape: (148670,)

num of negative label: 112031/num of positive label: 36639
% of negative label: 75.3554853030201/% of positive label: 24.64451469697989


# Data Cleaning

In [19]:
print('Start Removing Duplicates')
duplicates = eda_df.duplicated()
# 중복제거시 ~duplicates해주면 된다.
eda_df = eda_df[~duplicates]
print('# of data after removing duplicates: {}\n'.format(eda_df.shape[0]))


print('Start Handling Missing Values')
print('features / # of missing values')
print(eda_df.isnull().sum())

if missing_value_flag == 'remove':
  print('Remove missing values\n')
  eda_df = eda_df.dropna()
elif missing_value_flag == 'replace':
  print('Replace missing values with mode\n')
   # .mode()는 최빈값을 의미한다.
  for feature_name in eda_df.columns:
    if eda_df[feature_name].isnull().sum() > 0:
      eda_df[feature_name].fillna(eda_df[feature_name].mode().iloc[0], inplace=True)

print('Start Handling Outliers using IQR')
outlier_dict = {}

for feature_name in eda_df.columns:
  if np.issubdtype(eda_df[feature_name].dtype, np.number):
    Q1 = eda_df[feature_name].quantile(0.25)
    Q3 = eda_df[feature_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    num_outlier = ((eda_df[feature_name] < lower_bound) | (eda_df[feature_name] > upper_bound)).sum()
    outlier_dict[feature_name] = num_outlier

    eda_df = eda_df[(eda_df[feature_name] >= lower_bound) & (eda_df[feature_name] <= upper_bound)]


print('Total # of outliers: {}'.format(sum(outlier_dict.values())))
print('Number of outliers for each feature')
for feature_name, num_outlier in outlier_dict.items():
  print('{}: {}'.format(feature_name, num_outlier))


Start Removing Duplicates
# of data after removing duplicates: 68443

Start Handling Missing Values
features / # of missing values
ID                           0
year                         0
loan_limit                   0
Gender                       0
approv_in_adv                0
loan_type                    0
loan_purpose                 0
Credit_Worthiness            0
open_credit                  0
business_or_commercial       0
loan_amount                  0
rate_of_interest             0
Interest_rate_spread         0
Upfront_charges              0
term                         0
Neg_ammortization            0
interest_only                0
lump_sum_payment             0
property_value               0
construction_type            0
occupancy_type               0
Secured_by                   0
total_units                  0
income                       0
credit_type                  0
Credit_Score                 0
co-applicant_credit_type     0
age                          0
s

In [22]:
numerical_features = eda_df.select_dtypes(include=[np.number])
numerical_array = numerical_features.values
print('numerical features shape: {}'.format(numerical_array.shape))

if scaling_flag == 'standardization':
  print('Standardization of numeircal features\n')
  feature_mean = np.mean(numerical_array, axis=0)
  # 표준편차 np.std
  feature_std = np.std(numerical_array, axis=0)
  # 분모가 0이면 안되므로 0일 경우 1e-5 대체한다.
  feature_std = np.where(feature_std == 0, 1e-5, feature_std)

  scaled_data = (numerical_array - feature_mean) / feature_std
elif scaling_flag == 'min-max':
  print('Min-max normalization of numeircal features\n')
  data_min = np.min(numerical_array, axis=0)
  data_max = np.max(numerical_array, axis=0)

  data_max = np.where(data_max == data_min, data_min + 1e-5, data_max)

  scaled_data = (numerical_array - data_min) / (data_max - data_min)

categorical_features = eda_df.select_dtypes(include=[object])
# 비숫자형 데이터의 값들
categorical_array = categorical_features.values
print('categorical features shape: {}'.format(categorical_array.shape))

categorical_encoded_list = []
print('One-hot encoding of categorical features\n')
num_encoded_dim = 0
print('categorical_features.columns:{}'.format(categorical_features.columns))
# 숫자가 아닌 feature의 컬럼을 원 핫 코딩을 통하여 숫자형식의 배열로 만들어 준다
# enumerate는 반복 가능한 객체를 순회하면서 각 요소에 인덱스를 자동으로 할당
for i, feature_name in enumerate(categorical_features.columns):
    # np.unique 함수는 해당 피처에서 유일한 값들(고유 값들)을 찾아서 배열로 반환
    unique_values = np.unique(categorical_array[:, i])
    print('{}: {}'.format(feature_name, unique_values))
    # num_encoded_dim은 원-핫 인코딩된 후의 전체 데이터에서의 열 개수(차원 수).
    # 카테고리형 피처의 고유 값의 개수에 따라 열이 추가되므로, 각 피처의 고유 값 개수를 모두 더한 값이 최종 dim
    num_encoded_dim += len(unique_values)

    encoded_array = np.zeros((categorical_array.shape[0], len(unique_values)))
  #  np.where(unique_values == value)는 현재 처리 중인 값(value)이 unique_values 배열에서 몇 번째 위치에 있는지를 반환.
  #  현재 행(j)에서 value가 고유 값 배열 unique_values에서 몇 번째 위치에 있는지를 기반으로 해당 인덱스에 1을 설정
    for j, value in enumerate(categorical_array[:, i]):
        encoded_array[j, np.where(unique_values == value)[0]] = 1
    categorical_encoded_list.append(encoded_array)

categorical_encoded_array = np.hstack(categorical_encoded_list)
print('\none-hot encoded dim: {}'.format(num_encoded_dim))
print('categorical encoded array shape: {}'.format(categorical_encoded_array.shape))
# np.hstack을 통하여 기존 숫자형 shape과 비 숫자형을 one-hot인코드하여 나온 shape를 합쳐준다
processed_array = np.hstack((scaled_data, categorical_encoded_array))
print('processed array shape: {}'.format(processed_array.shape))

numerical features shape: (66570, 13)
Min-max normalization of numeircal features

categorical features shape: (66570, 21)
One-hot encoding of categorical features

categorical_features.columns:Index(['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose',
       'Credit_Worthiness', 'open_credit', 'business_or_commercial',
       'Neg_ammortization', 'interest_only', 'lump_sum_payment',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'credit_type', 'co-applicant_credit_type', 'age',
       'submission_of_application', 'Region', 'Security_Type'],
      dtype='object')
loan_limit: ['cf' 'ncf']
Gender: ['Female' 'Joint' 'Male' 'Sex Not Available']
approv_in_adv: ['nopre' 'pre']
loan_type: ['type1' 'type2' 'type3']
loan_purpose: ['p1' 'p2' 'p3' 'p4']
Credit_Worthiness: ['l1' 'l2']
open_credit: ['nopc']
business_or_commercial: ['b/c' 'nob/c']
Neg_ammortization: ['neg_amm' 'not_neg']
interest_only: ['int_only' 'not_int']
lump_sum_payment: ['lp